In [1]:
import pandas as pd
import os
import numpy as np
import requests
from bs4 import BeautifulSoup
import bs4.element
import warnings
warnings.filterwarnings('ignore')
import re
import time
from datetime import datetime

In [2]:
folders = os.listdir('D:\\big16\\final project\\DATA\\NEWS_DATA\\1. UPLOAD\\경제')

df_all = pd.DataFrame()
for i in range(0,len(folders)):
    if folders[i].split('.')[1] == 'csv':
        file = 'D:\\big16\\final project\\DATA\\NEWS_DATA\\1. UPLOAD\\경제\\'+folders[i]
        df= pd.read_csv(file,encoding='utf-8') 
        df_all = pd.concat([df_all, df])

In [ ]:
# df_all = pd.read_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\국제시장_merge.csv', index_col=False)

In [3]:
df_all.head()

,date,title,link,content
0,2022-06-30 01:13:10,"최저임금 인상에 경제계 ""현실외면한 결정…인상률 동의 어려워""",https://n.news.naver.com/mnews/article/001/001...,"[\n경총·대한상의·중기중앙회·소공연 반발…""부작용 완화 방안 마련해야""(서울=연합..."
1,2022-06-30 16:07:01,9620원…경제수장도 ‘6% 물가’ 본다는데 5% 오른 최저임금,https://n.news.naver.com/mnews/article/028/000...,[\n올해 시급 9160원보다 5%(460원) 인상 물가상승 전망 4.5%…내년 더...
2,2022-06-30 20:22:01,"尹 ""한국전 참전에 고마움…경제 안보 협력 강화""",https://n.news.naver.com/mnews/article/079/000...,[\n\n\n\n\n북대서양조약기구(NATO·나토) 정상회의 참석차 스페인 마드리드...
3,2022-06-30 18:18:47,"尹, 나토 일정 마지막날 '경제 외교'...연기된 나토 총장 면담",https://n.news.naver.com/mnews/article/052/000...,[\n\t\t\t[앵커]나토정상회의 참석차 스페인을 방문 중인 윤석열 대통령의 첫 ...
4,2022-06-30 18:11:02,치솟는 가계대출 금리… 한 총리 “경제전쟁 상황”,https://n.news.naver.com/mnews/article/022/000...,[\n5월 금리 4.14%… 8년4개월래 최고저신용 대출 비중 늘어 신용금리 급등 ...


In [4]:
df_all.drop('link', axis=1, inplace=True)
# df_all.drop('Unnamed: 0', axis=1, inplace=True)
# df_all.drop('linkcontent', axis=1, inplace=True)

In [5]:
import re

def clean_text(d):
  text = re.sub(r'\([^)]*\)', '', str(d))
  text = re.sub(r'\[[^]]*\]', '', text)
  text = re.sub(r'\<[^>]*\>', '', text)
  pattern = r'[^가-힣0-9a-zA-Z\s]'
  text = re.sub(pattern, ' ', text)
  text = re.sub(r'사진', ' ', text)
  text = re.sub(r'.*뉴스', ' ', text)
  text = re.sub("\n", ' ', text)
  text = re.sub("  +", " ", text)
  return text

def text_clean(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern, '', text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern, '', text)
    pattern = '[a-zA-Z0-9]'    # 숫자와 알파벳 제거
    text = re.sub(pattern, '', text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern, '', text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern, '', text)
    pattern = '[^\w\s]'         # 특수기호제거
    text = re.sub(pattern, '', text)
    pattern = '[\n]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\t]'            # \n제거
    text = re.sub(pattern, '', text)
    pattern = '[\']'           
    text = re.sub(pattern, '', text)
    pattern = '[\"]'            
    text = re.sub(pattern, '', text)
    return text  

In [6]:
df_all.dropna(axis=0, inplace=True)

In [7]:
df_all['content'] = df_all['content'].apply(text_clean)
df_all['content'] = df_all['content'].apply(clean_text)
df_all

,date,title,content
0,2022-06-30 01:13:10,"최저임금 인상에 경제계 ""현실외면한 결정…인상률 동의 어려워""",김주형 기자 공익위원이 내년도 최저임금 원을 제시한 가운데 일 오후 세종시 정부세...
1,2022-06-30 16:07:01,9620원…경제수장도 ‘6% 물가’ 본다는데 5% 오른 최저임금,올 하반기 물가상승률이 대까지 치솟을 것으로 전망되는 가운데 내년 최저임금이 올해...
2,2022-06-30 20:22:01,"尹 ""한국전 참전에 고마움…경제 안보 협력 강화""",
3,2022-06-30 18:18:47,"尹, 나토 일정 마지막날 '경제 외교'...연기된 나토 총장 면담",가 됩니다카카오톡 검색해 채널 추가전화 메일
4,2022-06-30 18:11:02,치솟는 가계대출 금리… 한 총리 “경제전쟁 상황”,신용대출 금리가 치솟으면서 월 은행권 가계대출 평균금리가 년개월 만에 최고 수준을...
...,...,...,...
2192,2022-06-16 12:20:37,경제계 한국전 참전국 초청 감사행사 기념촬영,가치나 화제성이 있다고 판단되는 또는 영상을 뉴시스 영상부로 보내주시면 적극 반영...
2193,2022-06-24 11:36:06,"경제보다 외교에 승부수 … 기시다, 참의원선거 전략",글로벌 피플기시다 후미오 일본 총리가 과감한 외교 행보를 보이고 있다기시다 총리는...
2194,2022-06-23 22:06:12,"윤 대통령, '한국 경제학계 거목' 조순 전 부총리 빈소 조문",
2195,2022-06-16 14:07:02,[尹정부 경제정책] 법인세 최고세율 22%로 인하…1주택자 14억까지 종부세 비과세,에 몰린 형벌규정 행정제재로 전환정년 연장 논의 가시화증권거래세 로 인하연금저축 연...


In [8]:
df_all['title'] = df_all['title'].apply(text_clean)
df_all['title'] = df_all['title'].apply(clean_text)
df_all

,date,title,content
0,2022-06-30 01:13:10,최저임금 인상에 경제계 현실외면한 결정인상률 동의 어려워,김주형 기자 공익위원이 내년도 최저임금 원을 제시한 가운데 일 오후 세종시 정부세...
1,2022-06-30 16:07:01,원경제수장도 물가 본다는데 오른 최저임금,올 하반기 물가상승률이 대까지 치솟을 것으로 전망되는 가운데 내년 최저임금이 올해...
2,2022-06-30 20:22:01,한국전 참전에 고마움경제 안보 협력 강화,
3,2022-06-30 18:18:47,나토 일정 마지막날 경제 외교연기된 나토 총장 면담,가 됩니다카카오톡 검색해 채널 추가전화 메일
4,2022-06-30 18:11:02,치솟는 가계대출 금리 한 총리 경제전쟁 상황,신용대출 금리가 치솟으면서 월 은행권 가계대출 평균금리가 년개월 만에 최고 수준을...
...,...,...,...
2192,2022-06-16 12:20:37,경제계 한국전 참전국 초청 감사행사 기념촬영,가치나 화제성이 있다고 판단되는 또는 영상을 뉴시스 영상부로 보내주시면 적극 반영...
2193,2022-06-24 11:36:06,경제보다 외교에 승부수 기시다 참의원선거 전략,글로벌 피플기시다 후미오 일본 총리가 과감한 외교 행보를 보이고 있다기시다 총리는...
2194,2022-06-23 22:06:12,윤 대통령 한국 경제학계 거목 조순 전 부총리 빈소 조문,
2195,2022-06-16 14:07:02,정부 경제정책 법인세 최고세율 로 인하주택자 억까지 종부세 비과세,에 몰린 형벌규정 행정제재로 전환정년 연장 논의 가시화증권거래세 로 인하연금저축 연...


In [10]:
df_all.to_csv('D:\\big16\\final project\\DATA\\NEWS_DATA\\0. merge\\경제_merge.csv')

In [11]:
import pymysql
conn = pymysql.connect(
    host='34.64.240.96'
    , user='root'
    , password='tndusWkd1.'
    , db='final_project'
    , charset='utf8'
)
cur = conn.cursor()

In [12]:
NewsDate = df_all['date'].to_list()
NewsTItle = df_all['title'].to_list()
NewsContent = df_all['content'].to_list()

In [13]:
for date,title,content in zip(NewsDate, NewsTItle, NewsContent):
    sql = "INSERT INTO past_news_economy VALUES ({}, {}, {})".format("\""+date+"\"", "\""+title+"\"", "\""+content+"\"")
    # print(sql)
    cur.execute(sql)
conn.commit()
conn.close()